<div class="alert alert-block alert-info">
<b>Deadline:</b> March 23, 2022 (Wednesday) 23:00
</div>

# Exercise 2. Recommender system

In this exercise, your task is to design a recommender system.

## Learning goals:
* Practise tuning a neural network model by using different regularization methods.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import tools
import data

In [2]:
skip_training = False  # Set this flag to True before validation and submission

In [3]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

import tools, warnings
warnings.showwarning = tools.customwarn

In [4]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is ../data


In [5]:
# Select the device for training (use GPU if you have one)
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"

In [6]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## Ratings dataset

We will train the recommender system on the dataset in which element consists of three values:
* `user_id` - id of the user (the smallest user id is 1)
* `item_id` - id of the movie (the smallest item id is 1)
* `rating` - rating given by the user to the item (ratings are integer numbers between 1 and 5.

The recommender system need to predict the rating for any given pair of `user_id` and `item_id`.

We measure the quality of the predicted ratings using the mean-squared error (MSE) loss:
$$
  \frac{1}{N}\sum_{i=1}^N (r_i - \hat{r}_i)^2
$$
where $r_i$ is a real rating and $\hat{r}_i$ is a predicted one.

Note: The predicted rating $\hat{r}_i$ does not have to be an integer number.

In [7]:
trainset = data.RatingsData(root=data_dir, train=True)
testset = data.RatingsData(root=data_dir, train=False)

In [8]:
# Print one sample from the dataset
x = trainset[0]
print(f'user_id={x[0]}, item_id={x[1]}, rating={x[2]}')

user_id=1, item_id=1, rating=5


# Model

You need to design a recommender system model with the API described in the cell below.

Hints on the model architecture:
* You need to use [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html?highlight=embedding#torch.nn.Embedding) layer to convert inputs `user_ids` and `item_ids` into reasonable representations. The idea of the embedding layer is that we want to represent similar users with values that are close to each other. The original representation as integers is not good for that. By using the embedding layer, we can learn such useful representations automatically.

### Model tuning

In this exercise, you need to tune the architecture of your model to achieve the best performance on the provided test set. You will notice that overfitting is a severe problem for this data: The model can easily overfit the training set producing poor accuracy on the out-of-training (test) data.

You need to find an optimal combination of the hyperparameters, with some hyperparameters corresponding to the regularization techniques that we studied in the lecture.

The hyperparameters that you are advised to consider:
* Learning rate value and learning rate schedule (decresing the learning rate often has positive effect on the model performance)
* Number of training epochs
* Network size
* Weight decay
* Early stopping
* Dropout
* Increase amount of data:
  * Data augmentation
  * Injecting noise

You can tune the hyperparameters by, for example, grid search, random search or manual tuning. In that case, you can use `architecture` argument to specify the hyperparameters that define the architecture of your network. After you have tuned the hyperparameters, set the default value of this argument to the optimal set of the hyparameters so that the best architecture is used in the accuracy tests.

Note:
* The number of points that you will get from this exercise depends on the MSE loss on the test set:
  * below 1.00: 1 point
  * below 0.95: 2 points
  * below 0.92: 3 points
  * below 0.90: 4 points
  * below 0.89: 5 points
  * below 0.88: 6 points 

In [9]:
class RecommenderSystem(nn.Module):
    def __init__(self, n_users, n_items,
                 architecture={'nof_layers': 3,
                               'nodes': [372, 364],
                               'dropout_rates': [0.021, 0.174, 0.36]}
                                    # If you want to tune the hyperparameters automatically (e.g. using random
                                    # search), use this argument to specify the hyperparameters that define the
                                    # architecture of your network. After you have tuned the hyperparameters,
                                    # set the default value of this argument to the optimal set of the hyparameters
                                    # so that the best architecture is used in the accuracy tests.
                ):
        """
        Args:
          n_users: Number of users.
          n_items: Number of items.
        """
        super(RecommenderSystem, self).__init__()
        #self.n_users = n_users
        #self.n_items = n_items
        self.n_emb = 1000
        
        self.users_e = nn.Embedding(n_users, self.n_emb)
        self.items_e = nn.Embedding(n_items, self.n_emb)
        
        self.layers = nn.ModuleList()
        
        if architecture is None:
          self.layers.append(nn.Sequential(
                        nn.Dropout(p = 0.1),
                        nn.Linear(self.n_emb*2 ,1),
                        nn.ReLU()))

        if architecture is not None:
          dropout_rates = architecture["dropout_rates"]
          nodes = architecture["nodes"]
          nof_layers = architecture["nof_layers"]
          dropout_rates = architecture["dropout_rates"]
          self.layers.append(nn.Sequential(
                              nn.Dropout(p = dropout_rates[0]),
                              nn.Linear(self.n_emb*2,nodes[0]),
                              nn.ReLU()))
          if nof_layers > 2:
              for i in range(nof_layers-2):
                  self.layers.append(nn.Sequential(
                                      nn.Dropout(p = dropout_rates[i+1]),
                                      nn.Linear(nodes[i],nodes[i+1]),
                                      nn.ReLU()))
          self.layers.append(nn.Sequential(
                              nn.Dropout(p = dropout_rates[len(dropout_rates)-1]),
                              nn.Linear(nodes[len(nodes)-1],1),
                              nn.ReLU()))
        
    def forward(self, user_ids, item_ids):
        """
        Args:
          user_ids of shape (batch_size): User ids (starting from 1).
          item_ids of shape (batch_size): Item ids (starting from 1).
        
        Returns:
          outputs of shape (batch_size): Predictions of ratings.
        """
        corrected_user_ids = torch.tensor(list(map(lambda id: id - 1, user_ids)), dtype=torch.int32)
        corrected_item_ids = torch.tensor(list(map(lambda id: id - 1, item_ids)), dtype=torch.int32)
        features = torch.cat([self.users_e(corrected_user_ids), self.items_e(corrected_item_ids)], dim=1)
        x = features
        for layer in self.layers:
            x = layer(x)
        res = x
        return torch.reshape(res, (-1,))

You can test the shapes of the model outputs using the function below.

In [10]:
def test_RecommenderSystem_shapes():
    n_users, n_items = 100, 1000
    model = RecommenderSystem(n_users, n_items)
    batch_size = 10
    user_ids = torch.arange(1, batch_size+1)
    item_ids = torch.arange(1, batch_size+1)
    output = model(user_ids, item_ids)
    print(output.shape)
    assert output.shape == torch.Size([batch_size]), "Wrong output shape."
    print('Success')

test_RecommenderSystem_shapes()

torch.Size([10])
Success


In [11]:
# This cell is reserved for testing

## Train the model

You need to train a recommender system using **only the training data.** Please use the test set to select the best model: the model that generalizes best to out-of-training data.

**IMPORTANT**:
* During testing, the predictions are produced by `predictions = model(user_ids, item_ids)` with the `user_ids` and `item_ids` loaded from `RatingsData`.
* There is a size limit of 30Mb for saved models.

In [12]:
def compute_accuracy(model, testloader, criterion):
    model.eval()
    evals = 0
    total = 0
    with torch.no_grad():
        for user_ids, item_ids, ratings in testloader:
            user_ids, item_ids, ratings = user_ids.to(device), item_ids.to(device), ratings.to(device).type(torch.float32)
            output = model(user_ids, item_ids).to(device)
            #_, predicted = torch.max(outputs.data, 1)
            evals += 1
            total += criterion(output, ratings).to(device).item()
    return total / evals

In [13]:
# Create the model
# IMPORTANT: the default value of the architecture argument should define your best model.
model = RecommenderSystem(trainset.n_users, trainset.n_items)

In [14]:
if not skip_training:
  import time
  from ray import tune
  from ray.tune import JupyterNotebookReporter
  from ray.tune.schedulers import ASHAScheduler
  from ray.tune.stopper import TrialPlateauStopper
  gpu_count = 0
  if torch.cuda.is_available():
      gpu_count = torch.cuda.device_count()
      if gpu_count > 1:
          model = nn.DataParallel(model)

In [16]:
# Implement the training loop in this cell
if not skip_training:


    def training_loop(hyperparameters, tune_mode = True):
        model = RecommenderSystem(trainset.n_users,
                                  trainset.n_items, 
                                  {"nof_layers": hyperparameters["nof_layers"],
                                   "nodes": hyperparameters["nodes"],
                                   "dropout_rates": hyperparameters["dropout_rates"]})
        #Training
        trainloader = torch.utils.data.DataLoader(trainset,
                                                  batch_size=hyperparameters["batch_size"],
                                                  shuffle=True)
        testloader = torch.utils.data.DataLoader(testset)
        
        params = list(model.parameters())
        optimizer = torch.optim.Adam(params, lr=hyperparameters["lr"], weight_decay = hyperparameters["weight_decay"])
        criterion = nn.MSELoss()
        for i in range(hyperparameters["epochs"]):
            model.train()
            for user_ids, item_ids, ratings in trainloader:
                user_ids, item_ids, ratings = user_ids.to(device), item_ids.to(device), ratings.to(device).type(torch.float32)
                optimizer.zero_grad()
                output = model(user_ids, item_ids).to(device)
                loss = criterion(output, ratings)
                loss.backward()
                optimizer.step()
            training_error = loss.item()
            test_error = compute_accuracy(model, testloader, criterion)
            if tune_mode:
                tune.report(training_error = training_error, validation_error = test_error)
                torch.save(model.state_dict(), "./model.pth")
            else:
                print(f"During epoch {i+1} training error = {training_error} and validation error = {test_error}")
        #torch.save(model.state_dict(), "./model.pth")
    #Hyperparameters
    config = {
    "lr": 0.001,#tune.qloguniform(1e-5, 2e-2, 1e-6),
    "epochs": 14,
    "batch_size": 159,#tune.randint(100,500),
    "weight_decay": 0.004,#tune.quniform(0, 0.10, 0.001),
    "nof_layers": 3,#tune.randint(2,7), #min 2 layers by default
    "nodes": [372, 364],#tune.sample_from(lambda spec:[np.random.randint(1,100) for i in range(spec.config.nof_layers-1)]),
    "dropout_rates": [0.021, 0.174, 0.36]}#tune.sample_from(lambda spec:[np.round(np.random.uniform(0, 0.5),decimals=3) for i in range(spec.config.nof_layers)])}
    
    scheduler = ASHAScheduler(
        metric="validation_error",
        mode="min",
        grace_period = 3)
    
    reporter = JupyterNotebookReporter(
        metric_columns=["training_error", "validation_error", "training_iteration"],
        overwrite = True,
        print_intermediate_tables = True,
        sort_by_metric = True,
        metric = "validation_error",
        mode = "min")
    
    stopper = TrialPlateauStopper(metric = 'validation_error',
                                  std = 0.01,
                                  num_results= 3,
                                  metric_threshold = 1.1,
                                  mode = 'max')
    
    #hyperopt_search = HyperOptSearch(config, metric="validation_error", mode="min")
    
    result  = tune.run(run_or_experiment  = training_loop,
                       config = config,
                       #metric = "validation_error",
                       #mode = "min",
                       num_samples=10,
                       resources_per_trial = {"gpu": gpu_count/2, "cpu": (torch.get_num_threads()/2)},
                       verbose = 1,
                       scheduler = scheduler,
                       progress_reporter=reporter,
                       stop=stopper)
                       
    best_trial = result.get_best_trial("validation_error", "min", "last")
    if(best_trial):
        print("Best trial config: {}".format(best_trial.config))
        print("Best trial final validation error: {}".format(
            best_trial.last_result["validation_error"]))
    else:
        print("No best trial")
        
    #visualization
    dfs = result.trial_dataframes

    # Plot by epoch
    ax = None  # This plots everything on the same plot
    for d in dfs.values():
        ax = d.validation_error.plot(ax=ax, legend=False)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Validation error')
    ax.set_ylim(0.5,2)

2022-04-29 17:30:57,152	INFO trial_runner.py:803 -- starting training_loop_eff83_00000
2022-04-29 17:30:59,150	WARNING util.py:171 -- The `start_trial` operation took 1.998 s, which may be a performance bottleneck.
(pid=) C:\Users\Christian Segercrant\AppData\Local\Programs\Python\Python39\python.exe: can't find '__main__' module in 'C:\\Users\\Christian'
(pid=) 2022-04-29 17:31:00,097	INFO context.py:67 -- Exec'ing worker with command: "C:\Users\Christian Segercrant\AppData\Local\Programs\Python\Python39\python.exe" C:\Users\Christian Segercrant\AppData\Roaming\Python\Python39\site-packages\ray\workers/default_worker.py --node-ip-address=127.0.0.1 --node-manager-port=65337 --object-store-name=tcp://127.0.0.1:59051 --raylet-name=tcp://127.0.0.1:57726 --redis-address=None --storage=None --temp-dir=C:\Users\CHRIST~1\AppData\Local\Temp\ray --metrics-agent-port=61681 --logging-rotate-bytes=536870912 --logging-rotate-backup-count=5 --gcs-address=127.0.0.1:57584 --redis-password=524159000000

KeyboardInterrupt: 

In [ ]:
#{'lr': 8e-05, 'epochs': 5, 'batch_size': 336, 'weight_decay': 0.049, 'nof_layers': 6, 'nodes': [405, 347, 425, 396, 350], 'dropout_rates': [0.046, 0.017, 0.128, 0.01, 0.234, 0.177]} # 0.9414899543032127
#{'lr': 7.0e-05, 'epochs': 9, 'batch_size': 159, 'weight_decay': 0.004, 'nof_layers': 3, 'nodes': [372, 364], 'dropout_rates': [0.021, 0.174, 0.36]} #0.9162588702653133
#{'lr': 0.001, 'epochs': 15, 'batch_size': 100, 'weight_decay': 0.001, 'nof_layers': 2, 'nodes': [10], 'dropout_rates': [0.02, 0.02]}# 0.8571209062045327

In [ ]:
if not skip_training:
  df = result.results_df
  logdir = result.get_best_logdir("validation_error", mode="min")
  state_dict = torch.load(os.path.join(logdir, "model.pth"))

  model = RecommenderSystem(trainset.n_users,
                            trainset.n_items,
                            architecture={'nof_layers': 3,
                                          'nodes': [372, 364],
                                          'dropout_rates': [0.021, 0.174, 0.36]})
  model.load_state_dict(state_dict)

In [ ]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
# Set confirm=False if you do not want to be asked for confirmation before saving.
if not skip_training:
    tools.save_model(model, 'recsys.pth', confirm=True)

In [ ]:
# This cell loads your best model
if skip_training:
    model = RecommenderSystem(trainset.n_users, trainset.n_items)
    tools.load_model(model, 'recsys.pth', device)

Model loaded from recsys.pth.


In [ ]:
if not skip_training:
  testloader = torch.utils.data.DataLoader(testset)
  criterion = nn.MSELoss()
  compute_accuracy(model, testloader, criterion)

The next cell tests the accuracy of your best model. It is enough to submit .pth files.

**IMPORTANT**:
* During testing, the predictions are produced by `predictions = model(user_ids, item_ids)` with the `user_ids` and `item_ids` loaded from `RatingsData`.
* There is a size limit of 30Mb for saved models. Please make sure that your model loads in the cell above.

In [ ]:
# This cell tests the accuracy of your best model.

In [ ]:
# This cell is reserved for grading

In [ ]:
# This cell is reserved for grading

In [ ]:
# This cell is reserved for grading

In [ ]:
# This cell is reserved for grading

In [ ]:
# This cell is reserved for grading

In [ ]:
# This cell is reserved for grading